In [16]:
import numpy as np
import pandas
import pandas as pd
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [17]:
reviews = []
with open('data/yelp_academic_dataset_review.json', encoding="ansi") as fl:
    i=0
    for review in fl:
        reviews.append(json.loads(review))
        i+=1
        if i + 1 > 10000:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0    3.0       0      0     0   
1    5.0       1      0     1   
2    3.0       0      0     0   
3    5.0       1      0     1   
4    4.0       1      0     1   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm...  2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delici

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [18]:
business = []
with open('data/yelp_academic_dataset_business.json', encoding="ansi") as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())



              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

ELIMINAMOS LAS COLUMNAS INNECESARIAS

COMPROBAMOS

In [19]:
df_review = df_review.drop(["review_id", "user_id","business_id", "useful", "funny", "cool", "date","stars"], axis=1)
#df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

In [20]:
print(df_review.head())
#print(df_business.head())

                                                text
0  If you decide to eat here, just be aware it is...
1  I've taken a lot of spin classes over the year...
2  Family diner. Had the buffet. Eclectic assortm...
3  Wow!  Yummy, different,  delicious.   Our favo...
4  Cute interior and owner (?) gave us tour of up...


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

vectorizer = TfidfVectorizer()
lemma = WordNetLemmatizer()

X = []
for r in df_review["text"]: #Por cada review
    txt = word_tokenize(r.lower()) #Separa la review en palabras
    newtxt = ""
    for w in txt: #Por cada palabra en txt
        l = lemma.lemmatize(w) #se hace lo de quitar mayusculas y quitar raice y eso
        newtxt += l+" "
    X.append(newtxt)

X = vectorizer.fit_transform(X)
print(X)

  (0, 15651)	0.2104454480765181
  (0, 6204)	0.1085558893661828
  (0, 13811)	0.08076084077667292
  (0, 21482)	0.11805926971455105
  (0, 13709)	0.048597114963181025
  (0, 16403)	0.07746435520037842
  (0, 13800)	0.06881285426357246
  (0, 5895)	0.14191164888090443
  (0, 1245)	0.09584477145921888
  (0, 7906)	0.0397741275911931
  (0, 13787)	0.19787959829199261
  (0, 21294)	0.11570871267253331
  (0, 21441)	0.0869712126229421
  (0, 18434)	0.13848425307602263
  (0, 21568)	0.09891737138495944
  (0, 12004)	0.09493387853267408
  (0, 20014)	0.15421211532850956
  (0, 14887)	0.12846249133663337
  (0, 20886)	0.22486024628977294
  (0, 22021)	0.14533499114322485
  (0, 21300)	0.17080797695753283
  (0, 13891)	0.061558115464438116
  (0, 4377)	0.0801217498073576
  (0, 11640)	0.19381874015665979
  (0, 21070)	0.11652868546031336
  :	:
  (9999, 15870)	0.12767739259830438
  (9999, 21669)	0.061965815037874114
  (9999, 1435)	0.08000467750412456
  (9999, 8598)	0.05835544214372858
  (9999, 14824)	0.0460347461996901

In [22]:
X= X.toarray()
#X=pd.DataFrame(X)
#X=np.array(X, dtype=object)
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
from sklearn.decomposition import PCA

#Antes de aplicar el método de clasificación utilizamos PCA para reducir el número de atributos
print('Dim originally: ',X.shape)
#Como vamos a representar gráficamente los clusters, nos quedaremos con los 2 atributos más imporantes
pca = PCA(n_components=2)
pca.fit(X)
# Cambio de base a dos dimensiones PCA
X = pca.transform(X)
print('Dim after PCA: ',X.shape)
print(X)


Dim originally:  (10000, 22189)
Dim after PCA:  (10000, 2)
[[ 0.11003364  0.11166316]
 [-0.05171281  0.12324963]
 [-0.10889372 -0.06413692]
 ...
 [-0.02617894  0.06520898]
 [ 0.0820722  -0.08249784]
 [-0.00582602  0.15804222]]


In [24]:
#X = pd.DataFrame(X)
#print(X[0][0])

In [28]:
class Distance_computation_grid(object):

    def __init__(self):
        pass

    def compute_distance(self,samples):

        Distance_mat = np.zeros((len(samples),len(samples))) #Hace una matriz de 10000*10000
        for i in range(Distance_mat.shape[0]):
            for j in range(Distance_mat.shape[0]):
                if i!=j:#Solo calcula distancias en una mitad de la matriz
                    Distance_mat[i,j] = float(self.distance_calculate(samples[i],samples[j])) #Para cada pos en la matriz calcula las distancias

                else:
                    Distance_mat[i,j] = 10**4 #Pone un valor muy grande
        return Distance_mat


    def distance_calculate(self,sample1,sample2):

        dist = []
        for i in range(len(sample1)):
            for j in range(len(sample2)):
                    dist.append(abs(sample1[i][0]-sample2[j][0])+abs(sample1[i][1]-sample2[j][1]))
        return np.min(dist)


    def intersampledist(self,s1,s2):
        if str(type(s2[0]))!='<class \'list\'>':
            s2=[s2]
        if str(type(s1[0]))!='<class \'list\'>':
            s1=[s1]
        m = len(s1)
        n = len(s2)
        dist = []
        if n>=m:
            for i in range(n):
                for j in range(m):
                    if (len(s2[i])>=len(s1[j])) and str(type(s2[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s2[i],s1[j]))
                    else:
                        dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))
        else:
            for i in range(m):
                for j in range(n):
                    if (len(s1[i])>=len(s2[j])) and str(type(s1[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s1[i],s2[j]))
                    else:
                        dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))
        return min(dist)

    def centroide(self,sample):
        if(len(sample) == 1):
            return np.mean()
        else:
            sample1=sample[0]
            i=1
            while i < len(sample):
                dist.append(abs(s1[i][0]-s2[j][0])+abs(s1[i][1]-s2[j][1]))

        return np.mean(dist)

    def interclusterdist(self,cl,sample):
        if sample[0]!='<class \'list\'>':
            sample = [sample]
        dist   = []
        for i in range(len(cl)):
            for j in range(len(sample)):
                dist.append(np.linalg.norm(np.array(cl[i])-np.array(sample[j])))
        return min(dist)

In [29]:
dist = []
print(X[0])
print(X[1])
print((np.array(X[0])-np.array(X[1])))
dist.append(abs(X[0][0]-X[1][0])+abs(X[0][1]-X[1][1]))
print(dist)

[0.11003364 0.11166316]
[-0.05171281  0.12324963]
[ 0.16174644 -0.01158647]
[0.17333291304338522]


In [30]:
progression = [[i] for i in range(X.shape[0])]
samples     = [[list(X[i])] for i in range(X.shape[0])]
m = len(samples)
distcal  = Distance_computation_grid()

print('Sample size before clustering    :- ',m)
Distance_mat      = distcal.compute_distance(samples)
print(Distance_mat)
dist_mat = pd.DataFrame(Distance_mat)

Sample size before clustering    :-  10000
[[1.00000000e+04 1.73332913e-01 3.94727431e-01 ... 1.82666752e-01
  2.22122434e-01 1.62238721e-01]
 [1.73332913e-01 1.00000000e+04 2.44567455e-01 ... 8.35745127e-02
  3.39532470e-01 8.06793877e-02]
 [3.94727431e-01 2.44567455e-01 1.00000000e+04 ... 2.12060679e-01
  2.09326830e-01 3.25246843e-01]
 ...
 [1.82666752e-01 8.35745127e-02 2.12060679e-01 ... 1.00000000e+04
  2.55957957e-01 1.13186164e-01]
 [2.22122434e-01 3.39532470e-01 2.09326830e-01 ... 2.55957957e-01
  1.00000000e+04 3.28438278e-01]
 [1.62238721e-01 8.06793877e-02 3.25246843e-01 ... 1.13186164e-01
  3.28438278e-01 1.00000000e+04]]


In [35]:

def buscarMin(matrizDistancias):
    columnas=0
    filas=0
    x= 0
    y=0
    min =1000000000000000000000000000000000000000000
    columnas=len(matrizDistancias)
    filas=len(matrizDistancias)

    for i in range(0,len(matrizDistancias)-1):
        for j in range(0,len(matrizDistancias)-1):
            if matrizDistancias[i][j] < min:
                x=i
                y=j
                min = matrizDistancias[i][j]

    return x,y, min

print(dist_mat.min().min().())

1.8917517534883244e-05


In [ ]:
print('se han calculado las distancias')
sample_ind_needed = np.where(Distance_mat==Distance_mat.min())[0]
value_to_add      = samples.pop(sample_ind_needed[1])
samples[sample_ind_needed[0]].append(value_to_add)

print('Cluster Node 1                   :-',progression[sample_ind_needed[0]])
print('Cluster Node 2                   :-',progression[sample_ind_needed[1]])

progression[sample_ind_needed[0]].append(progression[sample_ind_needed[1]])
progression[sample_ind_needed[0]] = [progression[sample_ind_needed[0]]]
v = progression.pop(sample_ind_needed[1])
m = len(samples)

print('Progression(Current Sample)      :-',progression)
print('Cluster attained                 :-',progression[sample_ind_needed[0]])
print('Sample size after clustering     :-',m)
print('\n')


In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
def hierarchical_clustering(data,linkage,no_of_clusters):
    #first step is to calculate the initial distance matrix
    #it consists distances from all the point to all the point
    color = ['r','g','b','y','c','m','k','w']
    initial_distances = pairwise_distances(data,metric='euclidean')
    #making all the diagonal elements infinity
    np.fill_diagonal(initial_distances,sys.maxsize)
    clusters = find_clusters(initial_distances,linkage)

    #plotting the clusters
    iteration_number = initial_distances.shape[0] - no_of_clusters
    clusters_to_plot = clusters[iteration_number]
    arr = np.unique(clusters_to_plot)

    indices_to_plot = []
    fig = plt.figure()
    fig.suptitle('Scatter Plot for clusters')
    ax = fig.add_subplot(1,1,1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    for x in np.nditer(arr):
        indices_to_plot.append(np.where(clusters_to_plot==x))
    p=0

    print(clusters_to_plot)
    for i in range(0,len(indices_to_plot)):
        for j in np.nditer(indices_to_plot[i]):
               ax.scatter(data[j,0],data[j,1], c= color[p])
        p = p + 1

    plt.show()


def find_clusters(input,linkage):
    clusters = {}
    row_index = -1
    col_index = -1
    array = []


    for n in range(input.shape[0]):
        array.append(n)

    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, input.shape[0]):
        min_val = sys.maxsize

        for i in range(0, input.shape[0]):
            for j in range(0, input.shape[1]):
                if(input[i][j]<=min_val):
                    min_val = input[i][j]
                    row_index = i
                    col_index = j

        #once we find the minimum value, we need to update the distance matrix
        #updating the matrix by calculating the new distances from the cluster to all points

        #for Single Linkage
        if(linkage == "single" or linkage =="Single"):
            for i in range(0,input.shape[0]):
                if(i != col_index):
                    #we calculate the distance of every data point from newly formed cluster and update the matrix.
                    temp = min(input[col_index][i],input[row_index][i])
                    #we update the matrix symmetrically as our distance matrix should always be symmetric
                    input[col_index][i] = temp
                    input[i][col_index] = temp
        #for Complete Linkage
        elif(linkage=="Complete" or linkage == "complete"):
             for i in range(0,input.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = min(input[col_index][i],input[row_index][i])
                    input[col_index][i] = temp
                    input[i][col_index] = temp
        #for Average Linkage
        elif(linkage=="Average" or linkage == "average"):
             for i in range(0,input.shape[0]):
                if(i != col_index and i!=row_index):
                    temp = (input[col_index][i]+input[row_index][i])/2
                    input[col_index][i] = temp
                    input[i][col_index] = temp

        elif(linkage=="Centroid" or linkage =="centroid"):
            for i in range(0,input.shape[0]):
                if(i!=col_index and i!=row_index):
                    dist_centroid = cal_dist_from_centroid(i,row_index,col_index)
                    input[col_index][i] = dist_centroid
                    input[i][col_index] = dist_centroid

        #set the rows and columns for the cluster with higher index i.e. the row index to infinity
        #Set input[row_index][for_all_i] = infinity
        #set input[for_all_i][row_index] = infinity
        for i in range (0,input.shape[0]):
            input[row_index][i] = sys.maxsize
            input[i][row_index] = sys.maxsize

        #Manipulating the dictionary to keep track of cluster formation in each step
        #if k=0,then all datapoints are clusters

        minimum = min(row_index,col_index)
        maximum = max(row_index,col_index)
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()

    return clusters
clusters = hierarchical_clustering(X,1,2)
print(clusters)

In [ ]:
# Import the f
print(clusters)

In [ ]:
cluster and linkage functions
from scipy.cluster.hierarchy import fcluster, linkage

# Use the linkage() function
distance_matrix = linkage(X, method = 'ward', metric = 'euclidean')

In [ ]:
# Import the dendrogram function
from scipy.cluster.hierarchy import dendrogram

# Create a dendrogram
dn = dendrogram(distance_matrix)

# Display the dendogram
plt.show()